In [11]:
from modules.base import *

#### 1. Graph 스트리밍 사용하기 
- 여기서 말하는 스트리밍은 단계별로 출력하는 방식을 의미함. (토큰 스트리밍 X)
- 스트리밍 종류는 2가지

- stream_mode="updates"
    - 각 노드가 실행될때 마다 변경된 state만 업데이트
    - 내생각) `override`
    
- stream_mode="values"
    - 각 노드가 실행될때 마다 전체 state 업데이트
    - 내생각) `append`

In [22]:
@trace_function(enable_print=False)
def Node_call_model(state: MessagesState, 
               config: RunnableConfig):
    response = llm.invoke(state["messages"], config)
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node("Node_call_model", Node_call_model)
builder.add_edge(START, "Node_call_model")
builder.add_edge("Node_call_model", END)
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream({"messages": [HumanMessage(content="안녕 나는 창우라고해")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['Node_call_model']["messages"].pretty_print()

for chunk in graph.stream({"messages": [HumanMessage(content="나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['Node_call_model']["messages"].pretty_print()
    
for chunk in graph.stream({"messages": [HumanMessage(content="내 취미는 헬스장에서 운동하는거고, 돼지고기를 제일좋아해")]}, 
                          config, 
                          stream_mode="updates"):
    chunk['Node_call_model']["messages"].pretty_print()        

================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 어떻게 도와드릴까요?
================================== Ai Message ==================================

멋지네요, 창우님! 박사과정을 하고 계시다니 정말 대단합니다. 공부하시느라 많이 바쁘실 것 같은데, 어떤 분야를 연구하고 계신가요?
================================== Ai Message ==================================

운동을 취미로 하시다니 건강도 잘 챙기고 계시네요! 헬스장에서의 운동은 스트레스 해소에도 좋고 체력 관리에도 큰 도움이 되죠. 그리고 돼지고기를 좋아하신다니, 다양한 요리로 즐기실 수 있을 것 같아요. 혹시 좋아하는 돼지고기 요리가 있나요?


In [24]:
config = {"configurable": {"thread_id": "2"}}
for event in graph.stream({"messages": [HumanMessage(content="안녕 나는 창우라고해")]}, 
                          config, 
                          stream_mode="values"):
    pass

for m in event['messages']:
    m.pretty_print()

print(f"\n{RED}{'*'*80}{RESET}")

for event in graph.stream({"messages": [HumanMessage(content="나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.")]}, 
                          config, 
                          stream_mode="values"):
    pass
for m in event['messages']:
    m.pretty_print()

print(f"\n{RED}{'*'*80}{RESET}")
    
for event in graph.stream({"messages": [HumanMessage(content="내 취미는 헬스장에서 운동하는거고, 돼지고기를 제일좋아해")]}, 
                          config, 
                          stream_mode="values"):
    pass
for m in event['messages']:
    m.pretty_print()

================================ Human Message =================================

안녕 나는 창우라고해
================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?

********************************************************************************
================================ Human Message =================================

안녕 나는 창우라고해
================================== Ai Message ==================================

안녕하세요, 창우님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?
================================ Human Message =================================

나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어.
================================== Ai Message ==================================

와, 대단하시네요! 부산대학교에서 박사과정을 하고 계시다니 정말 멋집니다. 전공 분야는 무엇인가요? 연구나 공부하면서 특별히 흥미로운 점이 있으면 공유해 주세요.

********************************************************************************
================================ Human Message =================================

안녕 나는 창우라고해
=======================

#### 2. 토큰 스트리밍

하나씩 출력

In [25]:
node_to_stream = 'Node_call_model'
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="이순신 장군에 대해 알려줘")
async for event in graph.astream_events({"messages": [input_message]}, 
                                        config, 
                                        version="v2"):
    if event["event"] == "on_chat_model_stream" and event['metadata'].get('langgraph_node','') == node_to_stream:
        print(event["data"])

{'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content='이', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content='순', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content='신', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content=' 장', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content='군', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content='은', additional_kwargs={}, response_metadata={}, id='run-19c8c18d-af9a-4fe2-9320-489610ddc1d0')}
{'chunk': AIMessageChunk(content=' 조', additional_kwargs={}, response_metadata={}, id='run

이어 붙이기

In [31]:
node_to_stream = 'Node_call_model'
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="이순신 장군에 대해 알려줘")
async for event in graph.astream_events({"messages": [input_message]}, 
                                        config, 
                                        version="v2"):
    if event["event"] == "on_chat_model_stream" and event['metadata'].get('langgraph_node','') == node_to_stream:
        print(f"{YELLOW}{event['data']['chunk'].content}{RESET}", end="", flush=True)

이순신 장군은 조선 시대의 대표적인 군사 영웅으로, 임진왜란 기간 동안 일본의 침략으로부터 조선을 방어하는 데 큰 기여를 한 인물입니다. 그의 출생일은 1545년 4월 28일이며, 1598년 12월 16일에 전사했습니다. 이순신은 해군 장군으로서 뛰어난 전략과 전술로 유명하며, 그의 많은 해전 승리는 조선의 국난을 극복하는 데 큰 역할을 했습니다.

특히 명량 해전은 그의 가장 유명한 승리 중 하나로 꼽힙니다. 이 전투에서 이순신은 단 13척의 배로 133척의 일본 함대를 상대하여 승리를 거두었습니다. 그의 리더십과 전략적 통찰력은 오늘날까지도 많은 이들에게 존경받고 연구되고 있습니다.

이순신 장군은 그의 충성심과 애국심으로 인해 한국 역사에서 중요한 인물로 자리 잡고 있습니다. 그의 삶과 업적은 다양한 매체를 통해 기념되고 있으며, 대한민국에서는 그를 기리기 위한 여러 기념물과 동상이 세워져 있습니다. 이순신의 정신은 현대에도 많은 이들에게 영감을 주고 있습니다.